In [1]:
import numpy as np
from qiskit import *
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.primitives import BackendSamplerV2
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeBrisbane

In [2]:
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

In [3]:
backend = AerSimulator()

In [ ]:
def tomography(circuit, target, shots):
    bases = ['X', 'Y', 'Z']
    results = {}

    for base in bases:
        circ = circuit.copy()
        
        creg = ClassicalRegister(1)
        circ.add_register(creg)

        if base == 'X':
            circ.h(target)
        elif base == 'Y':
            circ.sdg(target)
            circ.h(target)
        elif base == 'Z':
            pass
        
        circ.measure(target, creg[0])

        transpiled = transpile(circ, backend)
        job = backend.run(transpiled,shots=shots)
        result = job.result()
        counts = result.get_counts()

        p0 = counts.get('0', 0) / shots
        p1 = counts.get('1', 0) / shots

        results[base] = p0 - p1


    rho = 0.5 * (I + results['X'] * X + results['Y'] * Y + results['Z'] * Z)
    return rho, results

In [6]:
def extend_circuit(base_circuit, num_extra_qubits, ext_name='ext'):
    qc_extended = base_circuit.copy()
    ext = QuantumRegister(num_extra_qubits, ext_name)
    qc_extended.add_register(ext)
    return qc_extended

def apply_swap_chain(circuit, start_qubit, num_swaps):
    for i in range(num_swaps):
        circuit.swap(start_qubit + i, start_qubit + i + 1)
    return circuit

def entenglament(n=0):
    Alice = QuantumRegister(1, 'Alice')
    Bob = QuantumRegister(1, 'Bob')
    circuit = QuantumCircuit(Alice, Bob)

    # Par emaranhado
    circuit.h(Alice)
    circuit.cx(Alice, Bob)

    circuit_ex = extend_circuit(circuit,n)
    circuit_swapado = apply_swap_chain(circuit_ex,start_qubit=1,num_swaps=n)

    circuit = circuit_swapado.copy()

    return circuit

In [7]:
circuit = entenglament(n=2)
rho_1 = tomography(circuit=circuit,target=0,shots=1000)
rho_2 = tomography(circuit=circuit,target=1,shots=1000)
rho_3 = tomography(circuit=circuit,target=2,shots=1000)
rho_4 = tomography(circuit=circuit,target=3, shots=1000)


In [8]:
print(rho_1)
print(rho_2)
print(rho_3)
print(rho_4)

(array([[ 0.501+0.j   , -0.01 +0.003j],
       [-0.01 -0.003j,  0.499+0.j   ]]), {'X': -0.020000000000000018, 'Y': -0.006000000000000005, 'Z': 0.0020000000000000018})
(array([[ 1.   +0.j   , -0.019-0.002j],
       [-0.019+0.002j,  0.   +0.j   ]]), {'X': -0.038000000000000034, 'Y': 0.0040000000000000036, 'Z': 1.0})
(array([[ 1.   +0.j   , -0.019-0.003j],
       [-0.019+0.003j,  0.   +0.j   ]]), {'X': -0.038000000000000034, 'Y': 0.006000000000000005, 'Z': 1.0})
(array([[ 0.524+0.j   , -0.008-0.016j],
       [-0.008+0.016j,  0.476+0.j   ]]), {'X': -0.016000000000000014, 'Y': 0.03200000000000003, 'Z': 0.04800000000000004})
